In [2]:
from cosmoNODE.loaders import Anode as A
from torch.utils.data import DataLoader

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from cosmoNODE.anode.conv_models import ConvODENet
from cosmoNODE.anode.models import ODENet
from cosmoNODE.anode.training import Trainer

In [3]:
a = A()
dataloader = DataLoader(a, batch_size=16)

In [34]:
item = a[0]
x = item[0]
y = item[1]
print(x.shape)
print(y)

torch.Size([1, 352, 2])
tensor(12)


In [35]:
device = torch.device('cuda:' if torch.cuda.is_available() else 'cpu')
conv_net = ConvODENet(device, img_size=(1, 352, 2), num_filters=32, output_dim=14, augment_dim=1)

In [42]:
epochs = 2
acc_list = []

for epoch in range(epochs):
    if epoch != 0:
        acc_list.append(correct/total)
    correct = 0
    total = 0
    for i, b_item in enumerate(dataloader):
        x_batch = b_item[0]
        y_batch = b_item[1]

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        y_pred = conv_net(x_batch)
        _, predicted = torch.max(y_pred.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()